In [1]:
%pip install tornado_helper --upgrade --no-cache-dir --find-links=. --force-reinstall --no-deps
%pip install folium

Looking in links: .
  Attempting uninstall: tornado_helper
    Found existing installation: tornado_helper 2.1.7
    Uninstalling tornado_helper-2.1.7:
      Successfully uninstalled tornado_helper-2.1.7
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tornado_helper import Combined

combined = Combined() 
df = combined.catalog()
df.head()

,type,start_time,end_time,lat,lon,episode_id,event_id,site_name,ef_number,category,az_min,az_max,rng_min,rng_max,filename,GOES_FILENAME,GOES_SATELLITE
0,train,2017-03-01 15:51:00+00:00,2017-03-01 16:06:00+00:00,37.377700,-81.008028,113548,679759,KFCX,-1.0,NUL,271.0,331.0,45932.0,105932.0,train/2017/NUL_170301_160534_KFCX_679759s_Q0.nc,ABI-L2-MCMIPC/2017/060/15/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
1,train,2017-03-01 15:59:00+00:00,2017-03-01 16:14:00+00:00,37.773164,-80.689291,113548,679759,KFCX,-1.0,NUL,306.0,366.0,60748.0,120748.0,train/2017/NUL_170301_161337_KFCX_679759s_V0.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
2,train,2017-03-01 16:00:00+00:00,2017-03-01 16:15:00+00:00,40.539268,-80.481657,114319,684965,KPBZ,-1.0,NUL,242.0,302.0,-7776.0,52224.0,train/2017/NUL_170301_161523_KPBZ_684965s_P7.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
3,train,2017-03-01 16:11:00+00:00,2017-03-01 16:26:00+00:00,37.022655,-81.461622,113548,679707,KFCX,-1.0,NUL,240.0,300.0,75564.0,135564.0,train/2017/NUL_170301_162553_KFCX_679707s_H1.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16
4,train,2017-03-01 16:11:00+00:00,2017-03-01 16:26:00+00:00,37.950221,-80.104371,113548,679759,KFCX,-1.0,NUL,-22.0,38.0,73712.0,133712.0,train/2017/NUL_170301_162553_KFCX_679759s_J0.nc,ABI-L2-MCMIPC/2017/060/16/OR_ABI-L2-MCMIPC-M3_...,noaa-goes16


In [3]:
from tornado_helper import GOES

entry = df[df["GOES_SATELLITE"] == "noaa-goes16"].iloc[50]
filename = entry["GOES_FILENAME"]
bucket = entry["GOES_SATELLITE"]

goes = GOES()
file = goes.download(filename, bucket)

Aria Download:   0%|          | 0.00/64.4M [00:00<?, ?B/s]

In [4]:
import xarray as xr 
import folium

broad_us_coords = [39.8283, -98.5795]
def make_map(lat_1, lat_2, lon_1, lon_2): 
    # Ensure lat_1 < lat_2 and lon_1 < lon_2
    bounds = [[min(lat_1, lat_2), min(lon_1, lon_2)],
              [max(lat_1, lat_2), max(lon_1, lon_2)]]
    
    m = folium.Map(location=broad_us_coords, zoom_start=4)
    folium.Rectangle(bounds=bounds, color="blue", fill=True, fill_opacity=0.2).add_to(m)
    
    return m

# Raw NOAA Coords
ds = xr.open_dataset(file[0])
[lat_min, lat_max, lon_min, lon_max] = combined._get_ds_coords(ds)

make_map(lat_min, lat_max, lon_min, lon_max)

In [5]:
# TorNet Coords
lat = entry["lat"]
lon = entry["lon"]

make_map(lat + 1, lat - 1, lon + 1, lon - 1)

In [6]:
# Clipped Cooords
clipped = combined._clip_ds_to_coord(ds, lat, lon)

[lat_min, lat_max, lon_min, lon_max] = combined._get_ds_coords(clipped)
make_map(lat_min, lat_max, lon_min, lon_max)


In [7]:
print(ds)

<xarray.Dataset> Size: 480MB
Dimensions:                                 (y: 1500, x: 2500,
                                             number_of_time_bounds: 2,
                                             number_of_image_bounds: 2, band: 1)
Coordinates: (12/37)
    t                                       datetime64[ns] 8B ...
  * y                                       (y) float32 6kB 0.1265 ... 0.04259
  * x                                       (x) float32 10kB -0.07501 ... 0.0...
    y_image                                 float32 4B ...
    x_image                                 float32 4B ...
    band_wavelength_C01                     (band) float32 4B ...
    ...                                      ...
    band_id_C11                             (band) int8 1B ...
    band_id_C12                             (band) int8 1B ...
    band_id_C13                             (band) int8 1B ...
    band_id_C14                             (band) int8 1B ...
    band_id_C15         

In [8]:
chopped = combined._clean_goes_dataset(clipped)

In [9]:
print(chopped)

<xarray.Dataset> Size: 955kB
Dimensions:                 (y: 81, x: 92, number_of_time_bounds: 2,
                             number_of_image_bounds: 2, band: 1)
Coordinates: (12/37)
    t                       datetime64[ns] 8B ...
  * y                       (y) float32 324B 0.1027 0.1026 ... 0.09825 0.0982
  * x                       (x) float32 368B 0.008596 0.008652 ... 0.01369
    y_image                 float32 4B ...
    x_image                 float32 4B ...
    band_wavelength_C01     (band) float32 4B ...
    ...                      ...
    band_id_C11             (band) int8 1B ...
    band_id_C12             (band) int8 1B ...
    band_id_C13             (band) int8 1B ...
    band_id_C14             (band) int8 1B ...
    band_id_C15             (band) int8 1B ...
    band_id_C16             (band) int8 1B ...
Dimensions without coordinates: number_of_time_bounds, number_of_image_bounds,
                                band
Data variables: (12/36)
    CMI_C01           